In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm
import pandas as pd


# Define the neural network architecture
class ParallelNN(nn.Module):
    def __init__(self, input1_size, hidden_size, num_classes):
        super(ParallelNN, self).__init__()

        # Define the first pipeline
        self.pipeline1 = nn.Sequential(
            nn.Linear(input1_size, 2048),
            nn.BatchNorm1d(2048),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(2048, hidden_size)
        )

        # Define the final classification layer
        self.classification = nn.Sequential(
            nn.Linear(hidden_size, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, num_classes)
        )

    def forward(self, x1):
        out1 = self.pipeline1(x1)

        # Apply the final classification layer
        final_out = self.classification(out1)
        return final_out

# Define the input sizes, hidden size, and number of classes
#input1_size = 10  # Change this to match the number of features in your first dataset
#input2_size = 8   # Change this to match the number of features in your second dataset
#hidden_size = 64
#num_classes = 3   # Change this to match the number of classes in your classification task

# Initialize the model
#model = ParallelNN(input1_size, input2_size, hidden_size, num_classes)

# Define your loss function and optimizer
#criterion = nn.CrossEntropyLoss()
#optimizer = optim.Adam(model.parameters(), lr=0.001)


In [8]:
# Assuming you have data1 and data2 as torch Tensors
# Replace this with your actual data
# data1 = torch.randn(10000, 10)  # Example: 100 samples, 10 features
# data2 = torch.randn(10000, 8)   # Example: 100 samples, 8 features
# labels = torch.randint(0, 3, (10000,))  # Example labels (0, 1, or 2)

device = "cpu"

# Load the data from a CSV file
csv_name = 'preprocessed_data.csv'
data = pd.read_csv(csv_name)

# Normalize the data within each column except the first
data.iloc[:, 1:] = (data.iloc[:, 1:] - data.iloc[:, 1:].mean()) / data.iloc[:, 1:].std()


# The first column is irrelevant, the second column is the label
labels = torch.tensor(data.iloc[:, 0].values, dtype=torch.long)
data = torch.tensor(data.iloc[:, 1:].values, dtype=torch.float32)

# Define model, loss function, and optimizer
model = ParallelNN(input1_size=data.shape[1], hidden_size=64, num_classes=3)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)
k_folds = 5

# Prepare data loaders
dataset = TensorDataset(data, labels)
train_set, val_set = torch.utils.data.random_split(dataset, [int(0.8 * len(dataset)), len(dataset) - int(0.8 * len(dataset))])
sampler = torch.utils.data.RandomSampler(train_set, replacement=True, num_samples=1000)
train_loader = DataLoader(train_set, batch_size=32, sampler=sampler)
val_loader = DataLoader(val_set, batch_size=16, shuffle=False)

# Training loop
def train(model, dataloader, criterion, optimizer):
    pbar = tqdm(dataloader)
    correct = 0

    for (data, labels) in pbar:
        model.train()

        optimizer.zero_grad()
        outputs = model(data)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        correct += (outputs.argmax(dim=1) == labels).float().sum()
        accuracy = correct / 1000 # Used in sampling

        
        # Validation loop
        val_loss = 0
        val_accuracy = 0
        model.eval()

        with torch.no_grad():
            for (data, labels) in val_loader:

                outputs = model(data)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                val_accuracy += (outputs.argmax(dim=1) == labels).float().sum()

        val_loss /= len(val_loader)
        val_accuracy /= len(val_set)
         
        pbar.set_description(f"Loss: {loss.item():.4f}, Accuracy: {accuracy:.4f}, Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}")
        

In [9]:
# Main function
def main():
    num_epochs = 30

    for epoch in range(num_epochs):
        print(f"Epoch {epoch+1}/{num_epochs}")
        train(model, train_loader, criterion, optimizer)

    print("Finished training")

if __name__ == '__main__':
    main()

Epoch 1/30


Loss: 1.1540, Accuracy: 0.5580, Val Loss: 1.0592, Val Accuracy: 0.4938: 100%|██████████| 32/32 [00:00<00:00, 52.87it/s]


Epoch 2/30


Loss: 0.4128, Accuracy: 0.6440, Val Loss: 0.9888, Val Accuracy: 0.4691: 100%|██████████| 32/32 [00:00<00:00, 80.44it/s]


Epoch 3/30


...

Epoch 29/30


Loss: 0.4305, Accuracy: 0.9650, Val Loss: 2.0974, Val Accuracy: 0.5062: 100%|██████████| 32/32 [00:00<00:00, 74.10it/s]


Epoch 30/30


Loss: 0.2370, Accuracy: 0.9480, Val Loss: 1.8804, Val Accuracy: 0.5062: 100%|██████████| 32/32 [00:00<00:00, 64.12it/s]

Finished training
